In [1]:
import numpy as np
from model_code import fastModel

In [3]:
import numba

@numba.jit(nopython=True)
def test_runtime(N, M, ß, T, reps):
    
    C = fastModel.create_connection_array_symmetrical(M, True)
    
    for i in range(reps):
        red = fastModel.reduced_model(M=M, N=N,T=T, ß=ß, C=C)
        exp = fastModel.explicit_model(M=M, N=N,T=T, ß=ß, C=C)

# this took 00:04:17, in MATLAB: 00:04:28
# test_runtime(1000, 4, 2.0, 4000, 600)

# this took 00:06:54, in MATLAB: 00:15:15
# test_runtime(1000, 64, 7.0, 4000, 600)

# this took 00:00:46, in MATLAB: 00:01:03
#test_runtime(100, 16, 3.8, 4000, 600)

#==> Python is quite a bit faster for me



In [2]:
# number of ß steps
ß_steps = 10

# upper and lower bounds of ß
ß_info = (0, 6, ß_steps)

# values of m: (for 3: 4, 16, 64)
M_values = 3

# number of samples per parameter combinations
no_of_simulations = 10

# number of agents
N = 100

# number of base iterations
base_iterations = 3

# run the simulation
red, exp = fastModel.SystematicParameterAnalysis(ß_info, M_values, no_of_simulations, N, base_iterations)

In [ ]:
# * 5 agents * 6 ß samples * 5 no_samples = Faktor 150

# 6 minuten * 150 = 900 minuten / 60 = 15 stunden :o

# ==> erstmal nur für 100 agenten? dann ca. 15/5 std :)


# x = 20 minuten  mit nogil lock bei dem modellcode und ohne nogil lock bei sypaan
# x = 16 minuten ohne nogil lock bei dem modellcode und ohne nogil lock bei sypaan
# x = 18 minuten ohne nogil lock bei dem modellcode und  mit nogil lock bei sypaan
# x = 18 minuten ohne nogil lock bei dem modellcode und  mit parallel=True bei sypaan
# x = 16 minuten ohne nogil lock bei dem modellcode und  mit fastmath=True bei sypaan

# x = 6 minuten mit numba und early stop                                                <== use this version

# x = 18 minuten ohne numba bei sypaan

In [3]:
print(red.shape)

In [2]:
from model_code import utilities_visuals as us
import matplotlib.pyplot as plt

def consensrate_withrespect_M_against_beta(SyPaAn_data, Ms):

    fig = plt.figure(figsize=(13, 7))
    fontsize = 14
    plt.rc('xtick',labelsize=fontsize-1)
    plt.rc('ytick',labelsize=fontsize-1)

    number = len(Ms)+4
    cmap = plt.get_cmap('autumn')
    colors = [cmap(i) for i in np.linspace(0, 1, number)]
    colors.reverse()

    for i, (color, edgecolor, M) in enumerate(zip(colors[2:-2], colors[4:], Ms)):
        SyPaAn_data_M = SyPaAn_data[:,i,:]
        #consensus_rate = np.mean(SyPaAn_data_M, axis=1)
        consensus_rate = np.mean(np.where(SyPaAn_data_M<0.01, 1, 0), axis=1)

        plt.plot(consensus_rate, lw=3, color = color, marker="o", markeredgewidth=1, markeredgecolor=edgecolor,
                 label=fr"Consens Rate for $M = {M}$ at $T = {int(M/4)*1000}$")
                 #label=fr"Consens Rate for $M = {M}$ at $T = {M * 1000}$")

    plt.xticks(np.arange(0,SyPaAn_data.shape[0],1), np.linspace(0, 5.8, SyPaAn_data.shape[0]).round(2),rotation=90)
    plt.title(fr"Consens Rate at different M's with $T = M/4 * 1000$", fontsize=fontsize)
    plt.legend(fontsize=fontsize)
    plt.xlabel(r"$\beta$", fontsize=fontsize)
    plt.ylabel("Consens Rate", fontsize=fontsize)

    #plt.savefig(f"consensrate_withrespect_M_against_beta{SyPaAn_data[0]['model_type']}.svg", format="svg")
    plt.show()

In [16]:
consensrate_withrespect_M_against_beta(red, [4,16,64])

In [6]:
consensrate_withrespect_M_against_beta(red, [4,16,64])

In [7]:
consensrate_withrespect_M_against_beta(exp, [4,16,64])

### one 

In [10]:
red_ßs60_ßlim_0_6_Ms_3_noS_50_it_250 = red
exp_ßs60_ßlim_0_6_Ms_3_noS_50_it_250 = exp

In [11]:
%store red_ßs30_ßlim_0_6_Ms_3_noS_10_it_250
%store exp_ßs30_ßlim_0_6_Ms_3_noS_10_it_250

In [12]:
%store -r red_ßs30_ßlim_0_6_Ms_3_noS_10_it_250
%store -r exp_ßs30_ßlim_0_6_Ms_3_noS_10_it_250

#### two

In [ ]:
red_ßs30_ßlim_0_6_Ms_3_noS_10_it_250 = red
exp_ßs30_ßlim_0_6_Ms_3_noS_10_it_250 = exp

In [ ]:
%store red_ßs30_ßlim_0_6_Ms_3_noS_10_it_250
%store exp_ßs30_ßlim_0_6_Ms_3_noS_10_it_250

In [ ]:
%store -r red_ßs30_ßlim_0_6_Ms_3_noS_10_it_250
%store -r exp_ßs30_ßlim_0_6_Ms_3_noS_10_it_250

In [ ]:
# number of ß steps
ß_steps = 60

# upper and lower bounds of ß
ß_info = (2, 8, ß_steps)

# values of m: (for 3: 4, 16, 64)
M_values = 3

# number of samples per parameter combinations
no_of_simulations = 50

# number of agents
N = 100

# number of base iterations
base_iterations = 3

# run the simulation
red, exp = fastModel.SystematicParameterAnalysis(ß_info, M_values, no_of_simulations, N, base_iterations)


In [ ]:
import matplotlib.pyplot as plt
def consensrate_withrespect_M_and_iteration_against_beta(SyPaAn_data, Ms, its):

    fig = plt.figure(figsize=(13, 7))
    fontsize = 14
    plt.rc('xtick',labelsize=fontsize-1)
    plt.rc('ytick',labelsize=fontsize-1)

    number = len(Ms)+4
    cmap = plt.get_cmap('Purples')
    colors = [cmap(i) for i in np.linspace(0, 1, number)]
    #colors.reverse()

    for i, (color, edgecolor, M) in enumerate(zip(colors[2:-2], colors[4:], Ms)):
        SyPaAn_data_M = SyPaAn_data[:,i,:,0]
        consensus_rate = np.mean(np.where(SyPaAn_data_M>0.2, 1, 0), axis=1)
        #consensus_rate = np.mean(np.where(SyPaAn_data_M<0.01, 1, 0), axis=1)

        plt.plot(consensus_rate, lw=3, color = color, marker="o", markeredgewidth=1, markeredgecolor=edgecolor,
                 label=fr"Polarization Rate for $M = {M}$ at $T = {its[0]}$")
        #label=fr"Consens Rate for $M = {M}$ at $T = {M * 1000}$")

    number = len(Ms)+4
    cmap = plt.get_cmap('Blues')
    colors = [cmap(i) for i in np.linspace(0, 1, number)]
    #colors.reverse()

    for i, (color, edgecolor, M) in enumerate(zip(colors[2:-2], colors[4:], Ms)):
        SyPaAn_data_M = SyPaAn_data[:,i,:,1]
        consensus_rate = np.mean(np.where(SyPaAn_data_M>0.2, 1, 0), axis=1)
        #consensus_rate = np.mean(np.where(SyPaAn_data_M<0.01, 1, 0), axis=1)

        plt.plot(consensus_rate, lw=3, color = color, marker="o", markeredgewidth=1, markeredgecolor=edgecolor,
                 label=fr"Polarization Rate for $M = {M}$ at $T = {its[1]}$")
        #label=fr"Consens Rate for $M = {M}$ at $T = {M * 1000}$")

    number = len(Ms)+4
    cmap = plt.get_cmap('Greens')
    colors = [cmap(i) for i in np.linspace(0, 1, number)]
    #colors.reverse()

    for i, (color, edgecolor, M) in enumerate(zip(colors[2:-2], colors[4:], Ms)):
        SyPaAn_data_M = SyPaAn_data[:,i,:,2]
        consensus_rate = np.mean(np.where(SyPaAn_data_M>0.2, 1, 0), axis=1)
        #consensus_rate = np.mean(np.where(SyPaAn_data_M<0.01, 1, 0), axis=1)

        plt.plot(consensus_rate, lw=3, color = color, marker="o", markeredgewidth=1, markeredgecolor=edgecolor,
                 label=fr"Polarization Rate for $M = {M}$ at $T = {its[2]}$")
        #label=fr"Consens Rate for $M = {M}$ at $T = {M * 1000}$")

    plt.xticks(np.arange(0,SyPaAn_data.shape[0],1), np.linspace(0, 5.8, SyPaAn_data.shape[0]).round(2),rotation=90)
    plt.title(fr"Polarization Rate at different M's with different base iterations", fontsize=fontsize)
    plt.legend(fontsize=fontsize)
    plt.xlabel(r"$\beta$", fontsize=fontsize)
    plt.ylabel("Polarization Rate", fontsize=fontsize)

    #plt.savefig(f"consensrate_withrespect_M_against_beta{SyPaAn_data[0]['model_type']}.svg", format="svg")
    plt.show()

In [4]:
consensrate_withrespect_M_and_iteration_against_beta(red, Ms=[4,16,64], its=[250,1000,4000])

In [12]:
exp[5,0,0,:]

In [9]:
consensrate_withrespect_M_and_iteration_against_beta(exp, Ms=[4,16,64], its=[250,1000,4000])

In [4]:
#%store reds
#%store exps

%store -r reds
%store -r exps